In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBoost
from sklearn.metrics import mean_squared_error

RANDOM_STATE = 55
file_path="/Users/amangolani/Downloads/Final Data.csv"

df=pd.read_csv(file_path);
df.drop(['Farm_ID','Irrigation_Type','Fertilizer_Used(tons)', 'Pesticide_Used(kg)','Water_Usage(cubic meters)'],axis=1,inplace=True)
cat_var=['Soil_Type','Season']
df=pd.get_dummies(data=df,prefix=cat_var,columns=cat_var)
df=pd.get_dummies(data=df,prefix=['Crop_Type'],columns=['Crop_Type'])
var = [x for x in df.columns if x not in 'Yield(tons)']
X_train,X_test,Y_train,Y_test=train_test_split(df[var],df['Yield(tons)'],train_size = 0.9, random_state = RANDOM_STATE)

# Updated parameters for RandomForestRegressor
random_forest_algo=RandomForestRegressor(n_estimators=100, max_depth=10, min_samples_split=4).fit(X_train,Y_train)
print(f"Metrics train:\n\tMSE: {mean_squared_error(random_forest_algo.predict(X_train),Y_train):.4f}\nMetrics test:\n\tMSE: {mean_squared_error(random_forest_algo.predict(X_test),Y_test):.4f}")

# Load augmented dataset
file_path="/Users/amangolani/Downloads/augmented_agriculture_dataset_simple.csv"
df=pd.read_csv(file_path);
df.drop(['Farm_ID','Irrigation_Type','Fertilizer_Used(tons)', 'Pesticide_Used(kg)','Water_Usage(cubic meters)'],axis=1,inplace=True)
cat_var=['Soil_Type','Season']
df=pd.get_dummies(data=df,prefix=cat_var,columns=cat_var)
df=pd.get_dummies(data=df,prefix=['Crop_Type'],columns=['Crop_Type'])
var = [x for x in df.columns if x not in 'Yield(tons)']
X_train,X_test,Y_train,Y_test=train_test_split(df[var],df['Yield(tons)'],train_size = 0.8, random_state = RANDOM_STATE)

# Updated parameters for RandomForestRegressor
random_forest_algo=RandomForestRegressor(n_estimators=200, max_depth=30, min_samples_split=5).fit(X_train,Y_train)
print(f"Metrics train:\n\tMSE: {mean_squared_error(random_forest_algo.predict(X_train),Y_train):.4f}\nMetrics test:\n\tMSE: {mean_squared_error(random_forest_algo.predict(X_test),Y_test):.4f}")


In [19]:
import pandas as pd
from sdv.tabular import CTGAN

# Load the datasets
file_paths = [
    '/Users/amangolani/Downloads/Final Data/AEP_hourly.csv',    # Replace with your file path if needed
    '/Users/amangolani/Downloads/Final Data/COMED_hourly.csv',
    '/Users/amangolani/Downloads/Final Data/DEOK_hourly.csv'
]

# Read CSV files and select the second column (ignoring datetime)
dfs = [pd.read_csv(file).iloc[:, 1] for file in file_paths]

# Merge the selected columns into a single DataFrame
merged_df = pd.concat(dfs, axis=1)
merged_df.columns = ['AEP', 'COMED', 'DEOK']

# Fill NaN values using synthetic data
# 1️⃣ Train CTGAN on existing (non-NaN) data
ctgan = CTGAN()
ctgan.fit(merged_df.dropna())

# 2️⃣ Generate synthetic data for missing values
num_missing = merged_df.isna().sum().sum()
synthetic_data = ctgan.sample(num_missing)

# 3️⃣ Fill NaN values with synthetic data
merged_df = merged_df.fillna(pd.DataFrame(
    synthetic_data.values[:num_missing].reshape(-1, 3),
    columns=merged_df.columns
))

# Expand dataset to 40,000 rows
additional_samples = 40000 - len(merged_df)
if additional_samples > 0:
    augmented_data = ctgan.sample(additional_samples)
    merged_df = pd.concat([merged_df, augmented_data], ignore_index=True)

# Save the final merged and augmented dataset
merged_df.to_csv('merged_augmented_dataset.csv', index=False)

print("✅ Dataset merged, NaNs filled, and augmented to 40,000 rows!")


ModuleNotFoundError: No module named 'sdv.tabular'

In [24]:
import pandas as pd
import numpy as np

# Load the datasets
aep = pd.read_csv('/Users/amangolani/Downloads/Final Data/AEP_hourly.csv')
comed = pd.read_csv('/Users/amangolani/Downloads/Final Data/COMED_hourly.csv')
deok = pd.read_csv('/Users/amangolani/Downloads/Final Data/DEOK_hourly.csv')

# Drop datetime columns
aep = aep.drop(columns=['Datetime'])
comed = comed.drop(columns=['Datetime'])
deok = deok.drop(columns=['Datetime'])

# Rename columns for clarity
aep.columns = ['AEP_Load']
comed.columns = ['COMED_Load']
deok.columns = ['DEOK_Load']

# Merge datasets on index
merged_df = pd.concat([aep, comed, deok], axis=1)

# Fill NaN values with random sampling from existing non-null data
for column in merged_df.columns:
    merged_df[column].fillna(np.random.choice(merged_df[column].dropna()), inplace=True)

# Generate synthetic-like data to reach 40,000 rows
current_size = len(merged_df)
target_size = 40000

if current_size < target_size:
    additional_rows = merged_df.sample(target_size - current_size, replace=True).reset_index(drop=True)
    final_dataset = pd.concat([merged_df, additional_rows], ignore_index=True)
else:
    final_dataset = merged_df.copy()

# Export the final dataset
final_dataset.to_csv('/mnt/data/final_merged_dataset.csv', index=False)

print("✅ Dataset merged, cleaned, and expanded to 40,000 entries!")

ImportError: cannot import name 'SequenceNotStr' from 'pandas._typing' (/Users/amangolani/anaconda3/lib/python3.11/site-packages/pandas/_typing.py)

In [22]:
import pandas as pd
print(pd.__version__)


2.0.3


In [ ]:
import pandas as pd

# Load the dataset
file_path = "household_power_consumption.csv"
household_data = pd.read_csv(file_path, delimiter=';', low_memory=False)

# Print column names
print(household_data.columns)
